In [ ]:
from common import *
from snobedo.snotel import CsvParser

In [ ]:
client = start_cluster(5, 12)
client_ip_and_port(client)

# Comparison between SNOTEL and iSNOBAL (HRRR F06) variables

In [ ]:
year = 2021
water_year = f"wy{year}"
time=slice(f"{year -1}-10-01", f"{year}-07-31")

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites_2x2.json')

## SNOTEL

In [ ]:
schofield_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Schofield/usda-csv/{year}-Schofield-Pass.csv',
)

butte_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Butte/usda-csv/{year}-Butte.csv',
)

taylor_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Taylor/usda-csv/{year}-Upper-Taylor.csv',
)

## iSnobal 

In [ ]:
wy_snow = xr.open_mfdataset(
    (SNOBAL_DIR / f'{water_year}' / 'erw/*/snow.nc').as_posix(),
    parallel=True,
)

butte_snobal = wy_snow.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat).compute()
schofield_snobal = wy_snow.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat).compute()
taylor_snobal = wy_snow.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat).compute()

In [ ]:
client.shutdown()

In [ ]:
plot_data = {
    'Butte': {
        'snobal': butte_snobal,
        'snotel': butte_snotel_csv,
        'color_snobal': 'sandybrown',
        'color_snotel': 'skyblue',
    },
    'Schofield Pass': {
        'snobal': schofield_snobal,
        'snotel': schofield_snotel_csv,
        'color_snobal': 'goldenrod',
        'color_snotel': 'steelblue',
    },
    'Upper Taylor': {
        'snobal': taylor_snobal,
        'snotel': taylor_snotel_csv,
        'color_snobal': 'khaki',
        'color_snotel': 'dodgerblue',
    },
}

## Compare SNOTEL to iSnobal

In [ ]:
figure_opts = dict(figsize=(9,5), dpi=300, tight_layout=True)
plot_range = pd.date_range(start=f'{year - 1}-10-01', periods=11, freq='MS')
text_props = dict(facecolor='whitesmoke', alpha=0.5, pad=.5, boxstyle='round')

xTicks = mdates.DateFormatter('%d-%b')

In [ ]:
def snobal_max_swe(site):
    return site.where(site.specific_mass == site.specific_mass.max(), drop=True).time[0].values

         
def swe_numbers(site, max_snobal, min_snobal):
    snobal_min = int(min_snobal['specific_mass'].sum())
    snobal_max = int(max_snobal['specific_mass'].sum())
    
    max_swe_date_snotel = plot_data[site]['snotel'].loc[plot_data[site]['snotel']['SWE(mm)'].idxmax()].name.date()
    max_swe_date_snobal = snobal_max_swe(plot_data[site]['snobal'])
    swe_date_diff = (max_swe_date_snobal - np.datetime64(max_swe_date_snotel)).astype('timedelta64[D]')
    
    snotel_sum = int(plot_data[site]['snotel']['SWE(mm)'].sum())
    
    legend = {}
    legend['Snotel'] = {}
    legend['Snotel']['total  '] = f'{snotel_sum} mm'
    legend['Snotel']['max SWE'] = str(max_swe_date_snotel)
    legend['Snobal'] = {}
    legend['Snobal']['max SWE'] = np.datetime_as_string(max_swe_date_snobal, unit='D')
    legend['Snobal']['  diff '] = str(swe_date_diff)
    legend['Snobal'][''] = ''
    legend['Snobal']['max total   '] = f'{snobal_max} mm'
    legend['Snobal']['min total   '] = f'{snobal_min} mm'
    legend['Snobal']['% Snotel max'] = f'{snobal_max/snotel_sum:.2%}'
    legend['Snobal']['% Snotel min'] = f'{snobal_min/snotel_sum:.2%}'

    return legend


def depth_numbers(site, max_snobal, min_snobal):
    melt_start = plot_data[site]['snotel'].loc[plot_data[site]['snotel']['SWE(mm)'].idxmax()].name
    snobal_depths = min_snobal['thickness'].squeeze('x').squeeze('y').to_pandas()
    min_date = snobal_depths[(snobal_depths == 0) & (snobal_depths.index > melt_start)].index[0]
    
    snobal_depths = max_snobal['thickness'].squeeze('x').squeeze('y').to_pandas()
    max_date = snobal_depths[(snobal_depths == 0) & (snobal_depths.index > melt_start)].index[0]

    snotel_depth = plot_data[site]['snotel']['Depth(m)']
    snotel_date = snotel_depth[(snotel_depth == 0) & (snotel_depth.index > melt_start)].index[0]

    legend = {}
    legend['Snotel melt-out'] = snotel_date.strftime(LEGEND_DATE)
    legend['Snobal Min     '] = min_date.strftime('%Y-%m-%d')
    legend['Snobal Max     '] = max_date.strftime(LEGEND_DATE)
    legend['Snobal Min diff'] = f'{(min_date - snotel_date).days} days'
    legend['Snobal Max diff'] = f'{(max_date - snotel_date).days} days'
    
    return legend

def print_hash(hash):
    import json
    import re
    print(re.sub(r'[{,},"]*', '', json.dumps(hash, indent=4)))

In [ ]:
def plot_variable(snobal_var, snotel, title, label, save_figure=False):
    figure, ax = plt.subplots(1, 1, **figure_opts)
    figure.set_facecolor('lightgrey')
    
    for site_name in plot_data.keys():
        max_snobal = plot_data[site_name]['snobal'].coarsen(**COARSEN_OPTS).max()
        min_snobal = plot_data[site_name]['snobal'].coarsen(**COARSEN_OPTS).min()
        
        min_snobal[snobal_var].plot(ax=ax, c='slategrey', alpha=0.8, lw=1, ls='--')
        max_snobal[snobal_var].plot(ax=ax, c='slategrey', alpha=0.8, lw=1, ls='--')
        ax.fill_between(
            max_snobal.time, 
            min_snobal[snobal_var].data.flatten(), 
            max_snobal[snobal_var].data.flatten(),
            label=site_name,
            color=plot_data[site_name]['color_snobal'], alpha=0.7
        )
        plot_data[site_name]['snotel'][snotel].plot(
            ax=ax, 
            label=f'{site_name} Snotel ', 
            color=plot_data[site_name]['color_snotel'], alpha=0.8, lw=1.5
        )
        
        print(site_name)    
        if title == 'Snow Depth':
            print_hash(depth_numbers(site_name, max_snobal, min_snobal))
        elif title == 'SWE':
            print_hash(swe_numbers(site_name, max_snobal, min_snobal))

    ax.xaxis.set_major_formatter(xTicks)
    ax.set_xlim([plot_range[0], plot_range[-1]])
    ax.set_xlabel(f'Water Year {plot_range[-1].year}')
    ax.set_ylim(bottom=0)
    ax.set_ylabel(label)
    
    ax.legend(loc='upper left', borderaxespad=0.15)
    ax.set_title(None);

    if save_figure:
        filename = f'{year}_{title}_{site_name}'.replace(' ', '-')
        plt.savefig(f"{FIGURES_DIR}/{filename}.png")

## Depth 

In [ ]:
plot_variable('thickness', 'Depth(m)', 'Snow Depth', 'Snow Depth (m)')

## SWE

In [ ]:
plot_variable('specific_mass', 'SWE(mm)', 'SWE', 'SWE (mm)')

## Snow Density

In [ ]:
# plot_variable('snow_density', 'Density(kg/m3)', 'Snow Density', r'Snow Density ($kg/m^3$)')

## Temperature 

In [ ]:
# plot_variable('temp_surf', 'Air-T(C)', 'temp_lower', 'Temperature (C)')